In [1]:
from collections import Counter

In [2]:
import csv
import os
import argparse
import time
import logging

In [3]:
from helper import filehandle_fastq_gz, print_logger, join_path

In [4]:
def bc_dict_seq2id(bc_index_fpath):
    """ dict[barcode_seq] = barcode_id """
    out = dict()
    with open(bc_index_fpath, 'rt') as fin:
        freader = csv.reader(fin, delimiter='\t')
        next(freader)
        out = {row[1]: int(row[0]) for row in freader}
    return(out)


def bc_dict_id2seq(bc_index_fpath):
    out = dict()
    with open(bc_index_fpath, 'rt') as fin:
        freader = csv.reader(fin, delimiter='\t')
        next(freader)
        out = {row[0]: int(row[1]) for row in freader}
    return(out)

In [5]:

def demultiplexing(read1_fpath, read2_fpath, dict_bc_seq2id,
                   outdir,
                   len_umi=6, len_bc=6, len_tx=35, bc_qual_min=10,
                   is_gzip=True,
                   do_bc_rev_complement=False,
                   do_tx_rev_complement=False,
                   verbose=False):
    """
    Demultiplexing to fastq files based on barcode sequence.


    """
    if is_gzip:
        fh_umibc = filehandle_fastq_gz(read1_fpath)
        fh_tx = filehandle_fastq_gz(read2_fpath)
    else:
        fh_umibc = open(read1_fpath, 'rt')
        fh_tx = open(read2_fpath, 'rt')

    sample_counter = Counter()

    bc_fhout = dict()
    for bc_seq, bc_id in dict_bc_seq2id.items():
        bc_fhout[bc_seq] = join_path(outdir,
                                     'BC-{}-{}.fastq'.format(bc_id, bc_seq))
    bc_fhout['UNKNOWNBC_R1'] = join_path(outdir, 'UNKNOWNBC_R1.fastq')
    bc_fhout['UNKNOWNBC_R2'] = join_path(outdir, 'UNKNOWNBC_R2.fastq')

    for bc_seq, v in bc_fhout.items():
        bc_fhout[bc_seq] = open(v, 'w')

    i = 0
    while(True):
        if verbose and i % 1000000 == 0:
            print_logger('Processing {:,} reads...'.format(i))
        try:
            umibc_name = next(fh_umibc).rstrip()
            umibc_seq = next(fh_umibc).rstrip()
            next(fh_umibc)
            umibc_qualstr = next(fh_umibc).rstrip()
            tx_name = next(fh_tx).rstrip()
            tx_seq = next(fh_tx).rstrip()
            next(fh_tx)
            tx_qualstr = next(fh_tx).rstrip()
            i += 1
        except StopIteration:
            break

#         Quality check? or user should feed good files
#         if not (umibc_name and umibc_seq and umibc_qualstr and tx_name and tx_seq and tx_qualstr):
#             raise Exception('FastQError: Possible Broken Fastq. Check pair-{}.\n'.format(i+1))
#         if len(umibc_seq) != len(umibc_qualstr) or len(tx_seq) != len(tx_qualstr):
#             raise Exception('FastQError: Possible multi-line Fastq. Convert to 4-line please.\n')
#         if umibc_name.split()[0] != tx_name.split()[0]:
#             raise Exception('FastQError: Reads are not paired at pair-{}.\n'.format(i+1))

        sample_counter['total'] += 1

        if len(umibc_seq) < len_umi + len_bc:
            continue

        umibc_min_qual = min((ord(c)-33 for c in umibc_qualstr[:(len_umi + len_bc)]))
        if umibc_min_qual < bc_qual_min:
            continue

        sample_counter['qualified'] += 1

        umi, cell_bc = umibc_seq[0:len_umi], umibc_seq[len_umi:(len_umi + len_bc)]
        try:
            fhout = bc_fhout[cell_bc]
        except KeyError:
            fhout = bc_fhout['UNKNOWNBC_R1']
            fhout.write('{}\n{}\n{}\n{}\n'.format(umibc_name, umibc_seq,
                                                  "+", umibc_qualstr))
            fhout = bc_fhout['UNKNOWNBC_R2']
            fhout.write('{}\n{}\n{}\n{}\n'.format(tx_name, tx_seq,
                                                  "+", tx_qualstr))
            sample_counter['unknown'] += 1
            continue

#         if len(tx_seq) < len_tx:
#             continue
        if len(tx_seq) > len_tx:
            tx_seq, tx_qualstr = tx_seq[:len_tx], tx_qualstr[:len_tx]
        read_name = '@BC-{}_UMI-{}'.format(cell_bc, umi)
        fhout.write('{}\n{}\n{}\n{}\n'.format(read_name, tx_seq,
                                              "+", tx_qualstr))
        sample_counter[cell_bc] += 1
        sample_counter['saved'] += 1


    sample_counter['unqualified'] = sample_counter['total'] - sample_counter['qualified']
    for _, v in bc_fhout.items():
        v.close()
    fh_umibc.close()
    fh_tx.close()

    return(sample_counter)

In [6]:

def write_demultiplexing(stats, dict_bc_seq2id, stats_fpath):
    if stats_fpath is None:
        stats_fpath = 'demultiplexing.log'
    try:
        fh_stats = open(stats_fpath, 'w')
    except Exception as e:
        raise Exception(e)
    fh_stats.write('BC\tReads(#)\tReads(%)\n')
    for k, v in dict_bc_seq2id.items():
        fh_stats.write('{:03d}-{}\t{:,}\t{:06.2f}\n'.format(v, k, stats[k],
                                                            stats[k]/stats['total']*100))
    fh_stats.write('{}\t{}\t{:06.2f}\n'.format('saved',
                                               stats['saved'],
                                               stats['saved']/stats['total']*100))
    fh_stats.write('{}\t{}\t{:06.2f}\n'.format('unknown',
                                               stats['unknown'],
                                               stats['unknown']/stats['total']*100))
    fh_stats.write('{}\t{}\t{:06.2f}\n'.format('qualified',
                                               stats['qualified'],
                                               stats['qualified']/stats['total']*100))
    fh_stats.write('{}\t{}\t{:06.2f}\n'.format('unqualified',
                                               stats['unqualified'],
                                               stats['unqualified']/stats['total']*100))
    fh_stats.write('{}\t{}\t{:06.2f}\n'.format('total',
                                               stats['total'],
                                               stats['total']/stats['total']*100))




# Main Run

In [7]:
bc_index_fpath='/ifs/data/yanailab/refs/barcodes/barcodes_cel-seq_umis96.tab'
r1_fpath='/ifs/home/yy1533/Lab/cel-seq-pipe/demo/data/7_S1_L001_R1_001.fastq.1M.gz'
r2_fpath='/ifs/home/yy1533/Lab/cel-seq-pipe/demo/data/7_S1_L001_R2_001.fastq.1M.gz'
# r1_fpath='/ifs/home/yy1533/Lab/cel-seq-pipe/demo/data/r1.gz'
# r2_fpath='/ifs/home/yy1533/Lab/cel-seq-pipe/demo/data/r2.gz'
outdir='/ifs/home/yy1533/Lab/cel-seq-pipe/demo/bc_split_Y'

In [8]:
! mkdir {outdir}

In [9]:
bc_dict = bc_dict_seq2id(bc_index_fpath)

In [13]:
t0 = time.time()
foo = demultiplexing(read1_fpath=r1_fpath, read2_fpath=r2_fpath,\
                         outdir=outdir, dict_bc_seq2id=bc_dict,\
                         len_umi=6, len_bc=6, len_tx=35, bc_qual_min=10,\
                         do_bc_rev_complement=False, do_tx_rev_complement=False,verbose=False)
t1 = time.time()

In [14]:
print('Demultiplexing takes {} seconds.'.format(t1-t0))

Demultiplexing takes 30.812251091003418 seconds.


In [12]:
write_demultiplexing(stats=foo, dict_bc_seq2id=bc_dict, 
                     stats_fpath=os.path.join(outdir, 'demultiplexing.log'))